### Assistant evaluation 제작

[대화 데이터셋 파일](./templates/examples/dialog_generate_prompt_examples.yaml)로 부터 [assistant evaluation 파일](./evaluation_data/assistant_evaluation.yaml)을 제작합니다.

단계는 아래와 같습니다.
1. 수집된 대화 데이터셋을 기반으로 대화를 자릅니다.
2. 사람이 직접 annotation을 지정합니다.

Agent의 행동 단계는 아래와 같습니다.
1. 시작하기: 자기 소개로 대화를 시작하세요. 정중함, 존중, 전문적인 어조를 유지하세요.
2. 분석: 와인 추천을 위한 고객의 니즈를 파악합니다. 와인 데이터베이스 도구는 사용할 수 없습니다. 고객이 와인을 즐길 때, 와인과 함께 무엇을 먹을지, 원하는 가격대에 대해 물어봅니다. 한 번에 한 가지 질문만 하세요.
3. 가격대 확인: 고객이 선호하는 가격대를 묻습니다. 다시 한 번 말씀드리지만, 이를 위한 도구는 제공되지 않습니다. 그러나 고객의 선호도와 가격대를 알고 있다면 도구를 사용하여 가장 적합한 와인 세 가지를 검색하고 와인을 추천하세요. 각 와인 추천은 비비노 링크, 가격, 등급, 와인 유형, 풍미 설명 및 이미지가 포함된 목록 형식의 제품 카드 형태로 이루어져야 합니다. 데이터베이스에 있는 와인만 추천에 사용하세요. 데이터베이스에 적합한 와인이 없는 경우 고객에게 알려주세요. 추천을 한 후 고객이 추천한 와인을 좋아하는지 문의합니다.
4. 와인 추천: 고객의 요구와 가격대에 따라 가장 적합한 와인 3가지를 제안합니다. 추천하기 전에 고객이 와인을 즐길 상황, 와인과 함께 먹을 음식, 원하는 가격대를 파악해야 합니다. 각 와인 추천은 비비노 링크, 가격, 등급, 와인 유형, 풍미 설명 및 이미지가 포함된 목록 형식의 제품 카드 형태로 이루어져야 합니다. 데이터베이스에 있는 와인만 추천에 사용하세요. 데이터베이스에 적합한 와인이 없는 경우 고객에게 알려주세요. 추천을 한 후 고객이 추천한 와인을 좋아하는지 문의합니다.
5. 판매: 고객이 추천 와인을 승인하면 자세한 설명을 제공합니다. 비비노 링크, 가격, 등급, 와인 종류, 풍미 설명 및 이미지가 포함된 목록 형식의 제품 카드를 제공합니다.
6. 위치 제안: 고객의 위치와 상황에 따라 와인 바를 추천하세요. 추천하기 전에 항상 지도 도구를 사용하여 고객이 선호하는 위치의 지역을 찾아보세요. 그런 다음 와인 바 데이터베이스 도구를 사용하여 적합한 와인 바를 찾습니다. 와인 바의 이름, URL, 등급, 주소, 메뉴, 영업시간, 휴일, 전화, 요약, 이미지가 포함된 이미지를 목록 형식으로 제품 카드에 입력합니다. 추천에는 데이터베이스에 있는 와인 바만 사용하세요. 데이터베이스에 적합한 와인 바가 없는 경우 고객에게 알려주세요. 추천을 한 후 고객이 추천한 와인을 좋아하는지 문의합니다.
7. 대화 마무리하기: 고객의 의견에 적절하게 응답하여 대화를 마무리합니다.
8. 질문 답변: 이 단계에서는 고객의 문의에 답변합니다. 가능한 경우 검색 도구 또는 와인 데이터베이스 도구를 사용하여 구체적인 답변을 제공하세요. 가능한 자세하게 답변 설명합니다.
9. 기타 상황: 상황이 1~8단계 중 어느 단계에도 해당하지 않는 경우 이 단계를 사용합니다. 

In [19]:
from ruamel.yaml import YAML
import re

In [26]:
yaml = YAML()
yaml.indent(mapping=4, sequence=4)
yaml.default_flow_style = False
yaml.preserve_quotes = True

# ruamel.yaml을 사용하여 입력 YAML 파일을 파싱합니다.
with open('../model/templates/examples/dialog_generate_prompt_examples.yaml', 'r', encoding='utf-8') as f:
    data = yaml.load(f)

output_data = []

for item in data:
    # 대화를 줄바꿈('\n') 단위로 분할합니다.
    lines = item['conversation'].split('\n')

    for i in range(1, len(lines), 2):
        # 현재의 대화 단계를 가져와서 output_data에 추가합니다.
        output_item = {}
        
        conversation = '\n'.join(lines[:i]).strip() + '\n'
        conversation = conversation.replace('유저:', 'User:')
        conversation = conversation.replace('어시스턴트:', '이우선:')
        conversation = conversation.replace('\n', '<END_OF_TURN>\n')
        output_item['conversation'] = conversation
        output_item['answer'] = '\n'
        output_data.append(output_item)

# ruamel.yaml을 사용하여 output_data를 새로운 YAML 파일에 쓰기
with open('../data/legacy/evaluation_data/assistant_evaluation.yaml', 'w', encoding='utf-8') as f:
    yaml.dump(output_data, f)
